In [1]:
import numpy as np
import matplotlib.pyplot as plt


In [ ]:

# use bokeh to plot velocity profile
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, Range1d, Label,Panel, Tabs
from bokeh.io import output_notebook

In [ ]:
output_notebook()



In [ ]:
def plot_velocity_profile(velocity_profile):
    # create a figure
    p = figure(plot_width=400, plot_height=400)

    # create a column data source for the plots to share
    source = ColumnDataSource(data=dict(x=np.arange(len(velocity_profile)), y=velocity_profile))

    # plot the velocity profile
    p.scatter(x='x', y='y', width=0.5, source=source)

    # plot vertical lines
    p.vbar(x='x', width=0.01, bottom=0, top='y', source=source)


    # set the y range to 0 to 3
    p.y_range = Range1d(0, 3)

    # add a label to the plot
    label = Label(x=0, y=3, text='Lowest Value: ')

    # add the label to the plot
    p.add_layout(label)

    return p

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def scale_profile(vel, vel_change_amount, clip_range=None):
    # amount = 0 -> no change. >0 increases the values, <0 reduces the values

    new_vals =  vel + vel_change_amount * sigmoid(vel)
    if clip_range is not None:
        new_vals = np.clip(new_vals, clip_range[0], clip_range[1])

    return new_vals

def invert_profile(vel, inversion_amount):
    # amount = 0 -> no change. amount = 1 -> flat profile, amount = 2 -> inverted profile
    # interpolate between vel and 1-vel
    direction = 0.5 - vel
    return vel + direction * inversion_amount

def adjust_groove(groove_vel_profile, vel_change_amount, inversion_amount):
    """
    Adjust the groove velocity profile by scaling and inverting it
    :param groove_vel_profile: (np.array) velocity profile of the groove
    :param vel_change_amount: <0 makes the groove quieter, >0 makes it louder, 0 is no change
    :param inversion_amount: 0 is no change, 1 is flat profile, 2 is fully inverted profile
    :return:
    """

    def scale_profile(vel, vel_change_amount, clip_range=None):
        # amount = 0 -> no change. >0 increases the values, <0 reduces the values

        new_vals =  vel + vel_change_amount * sigmoid(vel)
        if clip_range is not None:
            new_vals = np.clip(new_vals, clip_range[0], clip_range[1])

        return new_vals

    def invert_profile(vel, inversion_amount):
        # amount = 0 -> no change. amount = 1 -> flat profile, amount = 2 -> inverted profile
        # interpolate between vel and 1-vel
        direction = 0.5 - vel
        return vel + direction * inversion_amount

    inverted_profile = invert_profile(groove_vel_profile, inversion_amount)
    scaled_inverted_profile = scale_profile(inverted_profile, vel_change_amount, clip_range=[0, 1])

    return scaled_inverted_profile


vel_change_amount = 1

inversion_amount = 2

# create a velocity profile
velocity_profile = np.random.rand(16)

# Apply softmax to the velocity profile
inverted_profile = invert_profile(velocity_profile, inversion_amount)
scaled_profile = scale_profile(velocity_profile, vel_change_amount, clip_range=[0, 1])
scaled_inverted_profile = scale_profile(inverted_profile, vel_change_amount, clip_range=[0, 1])
adjusted_profile = adjust_groove(velocity_profile, vel_change_amount, inversion_amount)

p1 = plot_velocity_profile(velocity_profile)
p2 = plot_velocity_profile(scaled_profile)
p3 = plot_velocity_profile(inverted_profile)
p4 = plot_velocity_profile(scaled_inverted_profile)
p5 = plot_velocity_profile(adjusted_profile)

show(Tabs(tabs=[Panel(child=p1, title="Original"),
                Panel(child=p2, title="Scaled"),
                Panel(child=p3, title="Inverted"),
                Panel(child=p4, title="Scaled Inverted"),
                Panel(child=p5, title="Adjusted")]))